In [1]:
from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from aap_core.agent import AgentMessage
from aap_core.orchestration import ReflectionAgent
from aap_langchain.chain import ChatCausalMultiTurnsChain
from langchain_ollama import ChatOllama

In [2]:
model = ChatOllama(model="qwen3:4b-thinking-2507-q4_K_M", base_url="192.168.55.1:11434")

In [ ]:
system_prompt_task = """/no_think You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Some requirements:
- The logic is clear and easy to understand.
- The function arguments and return values (if any) should be typed.
- If the function is too long (for example greater than 80 lines), split the logic into multiple smaller functions.
- All functions should have docstring explanation. In the explanation, there should be an simple example to illustrate the function and how to call it.
- The response should contain function with docstring explanation. And DO NOT contain explanation outside of the code
"""
user_prompt_task = "{query}"
task_chain = ChatCausalMultiTurnsChain(model=model, system_prompt=system_prompt_task, user_prompt_template=user_prompt_task)
task_chain.final_response_as_context("response")

system_prompt_reflection = """/no_think You are a excellent code reviewer and refactor.
Given a function implementation and it explanation, your task is to review and code and correct if contains any mistake.
Some note:
- For the implementation, check if the orignal query and suggested implementation are match.
- Is there any syntax error in the code.
- For the explanation, verify if the docstring follows Google style docstring.
- In the docstring, make sure to have an example to call the function.

Make sure the final output only contain full function code, inline code comment and docstring, nothing else."""
user_prompt_reflection = "Input query: {query}\n\nFunction implementation: {context_response}"
# Use the same llm for task and self-reflection
reflection_chain = ChatCausalMultiTurnsChain(model=model, system_prompt=system_prompt_reflection, user_prompt_template=user_prompt_reflection)  # use the same model for self-reflection

def state_callback(state: str):
    print(f"agent state: {state}")
reflection_skill = AgentSkill(
    id='reflection-skill',
    name="reflection skill",
    description="self-reflection skill",
    tags=['reflection']
)
reflection_card = AgentCard(
    name="reflection agent",
    description="self-reflection agent",
    skills=[reflection_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
reflection_agent = ReflectionAgent(card=reflection_card, chain_task=task_chain, chain_reflection=reflection_chain, state_change_callback=state_callback)

In [4]:
# Take a leetcode as an example. Source: https://leetcode.com/problems/palindrome-partitioning-ii/description/
query = """Write python function(s) to solve the following problem:
Given a string s, partition s such that every substring of the partition is a palindrome.
Return the minimum cuts needed for a palindrome partitioning of s.

Example 1:
Input: s = "aab"
Output: 1
Explanation: The palindrome partitioning ["aa","b"] could be produced using 1 cut.

Example 2:
Input: s = "a"
Output: 0

Example 3:
Input: s = "ab"
Output: 1

Constraints:
1 <= s.length <= 2000
s consists of lowercase English letters only."""

final_message = reflection_agent.execute(AgentMessage(query=query))
for response in final_message.responses:
    name, msg = response
    print(name)
    print(msg)

agent state: reflection agent:running


agent state: reflection agent:reflecting
agent state: reflection agent:idle
reflection agent


To solve this problem, we need to find the minimum number of cuts required to split a given string into palindromic substrings. The solution involves dynamic programming to efficiently compute the minimal cuts by leveraging precomputed palindrome information.

### Approach
1. **Precompute Palindromes**: First, we create a 2D array `is_pal` where `is_pal[i][j]` is `True` if the substring `s[i..j]` is a palindrome. This is done by checking substrings from the end of the string to the beginning, ensuring that we only check valid substrings that have already been processed.
2. **Dynamic Programming Setup**: We initialize a DP array `dp` where `dp[i]` represents the minimum cuts needed for the substring `s[0..i-1]`. The base case `dp[0]` is set to `-1` to handle edge cases correctly.
3. **DP Array Population**: For each position `i` from `1` to `n` (the length of the string), we check all possible